# Fraud AutoML

In [ ]:
import google
from google.cloud import aiplatform, bigquery

In [ ]:
credentials, project_id = google.auth.default(scopes=["htttps://googleapis.com/auth/cloud-platform"])

In [ ]:
bqclient = bigquery.Client(credentials=credentials, project=project_id)

In [ ]:
query_string = """SELECT * FROM `bigquery-public-data.ml_datasets.ulb_fraud_detection`"""

In [ ]:
dataframe = bqclient.query(query_string).result().to_dataframe()

In [ ]:
dataframe.info()

In [ ]:
dataframe.head()

In [ ]:
len(dataframe.columns.tolist())

In [ ]:
dataframe['Class'].value_counts()

## Vertex AI SDK

In [ ]:
BUCKET_NAME="gs://${project_id}"

In [ ]:
aiplatform.init(project=project_id, staging_bucket=BUCKET_NAME)

In [ ]:
# Creamos el dataset por codigo
dataset = aiplatform.TabularDataset.create(display_name='fraud_dataset_sdk', bq_source='bq://bigquery-public-data.ml_datasets.ulb_fraud_detection')

In [ ]:
# Training Job
job = aiplatform.AutoMLTabularTrainingJob(display_name='fraud_dataset_sdk', optimization_prediction_type="classification", optimization_objective='maximize-au-roc')

In [ ]:
model = job.run(dataset=dataset, target_column='Class', budget_milli_node_hours=100, model_display_name='fraud_model_sdk', disable_early_stopping=False)

In [ ]:
endpoint = model.deploy(machine_type='n1-standard-2')